In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Rio Grande Do Norte - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [6]:
data = pd.read_csv('2003_01_model_input_RN.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Rio Grande do Norte - IDH,Rio Grande Do Norte - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Norte - Desemprego,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande Do Norte - Consumo de Cimento (t)
0,2003,0.689488,338.269707,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,...,26.766463,21.887351,75.165436,4.896976,8.294170,3.164847e+07,1.998664e+06,9.097985,2.879532e+07,408.265000
1,2004,0.694345,350.872937,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,...,26.831552,21.801061,74.471505,5.121486,8.222820,3.213445e+07,2.027983e+06,9.137415,2.897550e+07,404.604000
2,2005,0.698728,368.606856,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,...,26.776125,21.751756,74.493209,5.757187,8.202615,3.257756e+07,2.041700e+06,9.167875,2.911598e+07,450.141000
3,2006,0.702649,400.635463,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,...,26.587320,21.621066,74.743047,6.059689,8.110415,3.296767e+07,2.059520e+06,9.185353,2.921015e+07,526.789000
4,2007,0.706119,425.000411,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,...,26.426482,21.587674,75.331406,5.585780,8.087322,3.330228e+07,2.079730e+06,9.191188,2.926087e+07,602.047000
5,2008,0.709150,443.220730,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,...,26.495543,21.686017,75.481661,5.066971,8.054622,3.355987e+07,2.079113e+06,9.184161,2.924771e+07,705.791000
6,2009,0.711755,467.794411,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,...,26.527542,21.707796,75.444194,4.900126,8.084471,3.376729e+07,2.091124e+06,9.172749,2.923115e+07,694.478000
7,2010,0.713945,615.433124,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,...,26.433934,21.583521,75.299709,4.741584,8.047148,3.462025e+07,2.094395e+06,9.216787,2.947144e+07,850.190000
8,2011,0.715731,928.296392,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,...,26.447359,21.511552,74.912036,4.560176,8.031985,3.538151e+07,2.094876e+06,9.249340,2.964338e+07,836.975000
9,2012,0.716652,1212.359182,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,...,26.509076,21.598394,75.029444,4.420323,12.625418,3.604796e+07,2.083213e+06,9.259759,2.976380e+07,961.431000


In [7]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Rio Grande do Norte - IDH,Rio Grande Do Norte - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Norte - Desemprego,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado
0,0.689488,338.269707,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,18.645227,...,2330.396397,26.766463,21.887351,75.165436,4.896976,8.294170,3.164847e+07,1.998664e+06,9.097985,2.879532e+07
1,0.694345,350.872937,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,17.130045,...,7472.792469,26.831552,21.801061,74.471505,5.121486,8.222820,3.213445e+07,2.027983e+06,9.137415,2.897550e+07
2,0.698728,368.606856,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,19.169803,...,14220.765319,26.776125,21.751756,74.493209,5.757187,8.202615,3.257756e+07,2.041700e+06,9.167875,2.911598e+07
3,0.702649,400.635463,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,21.685758,...,17267.573042,26.587320,21.621066,74.743047,6.059689,8.110415,3.296767e+07,2.059520e+06,9.185353,2.921015e+07
4,0.706119,425.000411,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,23.717622,...,18589.708713,26.426482,21.587674,75.331406,5.585780,8.087322,3.330228e+07,2.079730e+06,9.191188,2.926087e+07
5,0.709150,443.220730,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,22.830934,...,19153.860798,26.495543,21.686017,75.481661,5.066971,8.054622,3.355987e+07,2.079113e+06,9.184161,2.924771e+07
6,0.711755,467.794411,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,19.600176,...,19116.520066,26.527542,21.707796,75.444194,4.900126,8.084471,3.376729e+07,2.091124e+06,9.172749,2.923115e+07
7,0.713945,615.433124,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,18.602661,...,14819.658458,26.433934,21.583521,75.299709,4.741584,8.047148,3.462025e+07,2.094395e+06,9.216787,2.947144e+07
8,0.715731,928.296392,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,18.422753,...,8435.619250,26.447359,21.511552,74.912036,4.560176,8.031985,3.538151e+07,2.094876e+06,9.249340,2.964338e+07
9,0.716652,1212.359182,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,19.489672,...,4927.337152,26.509076,21.598394,75.029444,4.420323,12.625418,3.604796e+07,2.083213e+06,9.259759,2.976380e+07


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     404.604000
1     450.141000
2     526.789000
3     602.047000
4     705.791000
5     694.478000
6     850.190000
7     836.975000
8     961.431000
9     954.170000
10    886.165714
11    802.508571
12    730.261714
13    646.482000
14    485.012000
15    563.293000
16    744.863000
17    811.860000
18    866.416000
19           NaN
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [9]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rio Grande do Norte - IDH,Rio Grande Do Norte - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Norte - Desemprego,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado
0,-2.030201,-1.232653,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-0.538593,...,-0.863995,1.419499,-0.002104,-0.613277,0.306792,-0.872684,-1.665069,-0.692195,-2.494629,-2.054569
1,-1.619870,-1.198579,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.401411,...,-0.121226,1.953487,-0.273025,-1.147007,0.665710,-0.901222,-1.430595,-0.112443,-1.601249,-1.523255
2,-1.249614,-1.150633,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-0.239874,...,0.853452,1.498765,-0.427822,-1.130314,1.681989,-0.909304,-1.216803,0.158793,-0.911104,-1.109036
3,-0.918427,-1.064039,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,1.192833,...,1.293533,-0.050179,-0.838144,-0.938154,2.165591,-0.946181,-1.028589,0.511163,-0.515102,-0.831350
4,-0.625303,-0.998165,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,2.349874,...,1.484503,-1.369694,-0.942983,-0.485623,1.407965,-0.955417,-0.867143,0.910789,-0.382880,-0.681812
5,-0.369238,-0.948904,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,1.844951,...,1.565989,-0.803116,-0.634219,-0.370056,0.578558,-0.968496,-0.742862,0.898592,-0.542103,-0.720604
6,-0.149226,-0.882465,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,0.005201,...,1.560595,-0.540603,-0.565841,-0.398873,0.311828,-0.956558,-0.642789,1.136098,-0.800667,-0.769444
7,0.035737,-0.483304,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.562832,...,0.939956,-1.308557,-0.956023,-0.510002,0.058370,-0.971486,-0.231255,1.200775,0.197129,-0.060899
8,0.186658,0.362565,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,-0.665281,...,0.017845,-1.198419,-1.181980,-0.808177,-0.231642,-0.977551,0.136034,1.210287,0.934690,0.446116
9,0.264396,1.130569,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,-0.057725,...,-0.488892,-0.692097,-0.909327,-0.717874,-0.455221,0.859698,0.457580,0.979680,1.170769,0.801192


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     404.604000
1     450.141000
2     526.789000
3     602.047000
4     705.791000
5     694.478000
6     850.190000
7     836.975000
8     961.431000
9     954.170000
10    886.165714
11    802.508571
12    730.261714
13    646.482000
14    485.012000
15    563.293000
16    744.863000
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Rio Grande do Norte - IDH,Rio Grande Do Norte - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Norte - Desemprego,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado
17,1.048153,0.793617,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-1.927738,...,-0.902462,2.176045,2.06131,1.506174,-1.074664,0.939491,0.751768,-1.131139,-0.273722,0.314748


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    811.86
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [13]:
train_input.iloc[-(len(train_input)//5):]

,Rio Grande do Norte - IDH,Rio Grande Do Norte - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Norte - Desemprego,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado
14,1.293744,0.512794,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,-1.154662,...,-0.957655,-0.131761,1.325742,1.553440,-1.006916,1.063470,1.202019,-1.545731,0.805914,1.026058
15,1.300173,0.362687,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,0.163301,...,-0.968655,0.463204,1.823515,1.879973,-1.186350,1.030632,1.152827,-1.546308,0.673637,0.883841
16,1.222715,0.548932,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,-0.543475,...,-0.963762,1.252087,2.115319,1.872438,-1.193548,1.023618,1.030559,-1.412843,0.341727,0.696252


In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [17]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [18]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3035059338, 19320516, 2359451484, 1652898184, 3314187301, 1144787703, 2270412895, 2639407095, 2524977942, 3311736427, 160498878, 3852814971, 674725646, 3779873789, 2857580635, 3144500192, 3546222158, 547538487, 3131473740, 1643383587, 1475266134, 3226489857, 913769025, 1250866941, 3441672205, 1625240057, 4061176186, 442775483, 3881508394, 502347790, 1125954011, 2418601497, 329315096, 662827000, 2456184292, 460119371, 2413976705, 1415550865, 3513947017, 2430280543, 2914425351, 3992881395, 2189358828, 2089484208, 1958706339, 281734103, 2846998689, 2158618641, 1044296977, 204056017, 1183139460, 1350509160, 4069830407, 848988407, 3648931120, 3291171683, 1042209261, 3805333421, 1856902361, 1326807117, 931765654, 1381220628, 4106438825, 4065317401, 727831302, 1625947134, 1910204941, 1582078824, 3770698126, 1653630312, 669289265, 753555538, 2239178642, 2168684924, 875270814, 3004729475, 1780475023, 586776792, 3781961723, 3386607489, 3720509371, 1349678462, 1383940504, 268247370, 3144655443, 

val_loss: 12652.3642578125


Step: 88 ___________________________________________
val_loss: 12011.1337890625


Step: 89 ___________________________________________
val_loss: 12400.74609375


Step: 90 ___________________________________________
val_loss: 25774.255859375


Step: 91 ___________________________________________
val_loss: 12675.6142578125


Step: 92 ___________________________________________
val_loss: 12353.03125


Step: 93 ___________________________________________
val_loss: 24692.830078125


Step: 94 ___________________________________________
val_loss: 26258.8203125


Step: 95 ___________________________________________
val_loss: 11882.9931640625


Step: 96 ___________________________________________
val_loss: 13531.8271484375


Step: 97 ___________________________________________
val_loss: 27203.099609375


Step: 98 ___________________________________________
val_loss: 37165.703125


Step: 99 ___________________________________________
val_loss: 13957.7294921875


fina

In [19]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 475ms/step - loss: 513579.8125 - val_loss: 362837.2500
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 488461.0000 - val_loss: 116306.1953
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 488673.5312 - val_loss: 348882.1250
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 440236.0938 - val_loss: 532228.4375
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 400099.7812 - val_loss: 636489.8125
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 352207.9375 - val_loss: 424599.7500
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 382486.0000 - val_loss: 211502.1719
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 428435.9062 - val_loss: 416162.2500
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 392267.7188 - val_loss: 460026.5938
Epoch 10/

Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 127403.1484 - val_loss: 98613.4922
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 115536.2422 - val_loss: 124131.4609
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 107516.9609 - val_loss: 122945.5625
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 107131.6562 - val_loss: 121422.9609
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 108524.5859 - val_loss: 113346.6875
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 111288.7188 - val_loss: 121932.4297
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 115475.9453 - val_loss: 115167.5391
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 105228.5156 - val_loss: 118198.1875
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 107990.3984 - val_loss: 112145.2109
Ep

1/1 [==============================] - 0s 14ms/step - loss: 28529.2656 - val_loss: 29361.6250
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 26874.9102 - val_loss: 47275.6211
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 29384.8906 - val_loss: 29096.6270
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 28139.3652 - val_loss: 30041.9238
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 24119.2188 - val_loss: 28737.8652
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 26615.6621 - val_loss: 29835.0566
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 27692.0566 - val_loss: 29578.9688
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 24341.3086 - val_loss: 26745.2285
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 23842.5781 - val_loss: 26253.1504
Epoch 158/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 10487.1211 - val_loss: 14160.1338
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 10220.6523 - val_loss: 13957.5283
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 10503.6670 - val_loss: 14335.2568
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 13050.7363 - val_loss: 15826.6768
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 10827.3877 - val_loss: 14485.6846
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 10301.9600 - val_loss: 12588.3867
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 8300.4287 - val_loss: 13027.5557
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 8366.5771 - val_loss: 13443.7432
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 9803.2832 - val_loss: 12972.5420
Epoch 232/10000
1/1 [========

1/1 [==============================] - 0s 14ms/step - loss: 6938.5132 - val_loss: 11861.4268
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 6688.8047 - val_loss: 11846.4805
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 7252.7788 - val_loss: 11849.5781
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 6980.6348 - val_loss: 11867.5361
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 6635.2500 - val_loss: 11847.7256
Epoch 303/10000
1/1 [==============================] - 0s 15ms/step - loss: 6882.0762 - val_loss: 11865.1084
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 6619.5498 - val_loss: 11847.3037
Epoch 305/10000
1/1 [==============================] - 0s 14ms/step - loss: 6581.3413 - val_loss: 11848.1865
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 6563.7222 - val_loss: 11849.7148
Epoch 307/10000
1/1 [==============

...

1/1 [==============================] - 0s 14ms/step - loss: 6419.9897 - val_loss: 13357.2979
Epoch 899/10000
1/1 [==============================] - 0s 14ms/step - loss: 6109.4883 - val_loss: 13534.7998
Epoch 900/10000
1/1 [==============================] - 0s 14ms/step - loss: 5826.9570 - val_loss: 13560.8311
Epoch 901/10000
1/1 [==============================] - 0s 15ms/step - loss: 6066.0435 - val_loss: 13556.7803
Epoch 902/10000
1/1 [==============================] - 0s 14ms/step - loss: 8500.1797 - val_loss: 14298.9775
Epoch 903/10000
1/1 [==============================] - 0s 14ms/step - loss: 6569.3711 - val_loss: 12618.1445
Epoch 904/10000
1/1 [==============================] - 0s 14ms/step - loss: 6214.7383 - val_loss: 12660.3584
Epoch 905/10000
1/1 [==============================] - 0s 14ms/step - loss: 6003.3345 - val_loss: 12724.8477
Epoch 906/10000
1/1 [==============================] - 0s 13ms/step - loss: 5892.8477 - val_loss: 12831.2783
Epoch 907/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 5528.2759 - val_loss: 12896.9014
Epoch 974/10000
1/1 [==============================] - 0s 14ms/step - loss: 4840.5020 - val_loss: 14193.3154
Epoch 975/10000
1/1 [==============================] - 0s 14ms/step - loss: 4683.7236 - val_loss: 13154.1045
Epoch 976/10000
1/1 [==============================] - 0s 14ms/step - loss: 5606.5732 - val_loss: 13216.5518
Epoch 977/10000
1/1 [==============================] - 0s 14ms/step - loss: 5579.8579 - val_loss: 12793.2861
Epoch 978/10000
1/1 [==============================] - 0s 14ms/step - loss: 5547.7925 - val_loss: 13074.9424
Epoch 979/10000
1/1 [==============================] - 0s 14ms/step - loss: 5649.8174 - val_loss: 12891.0547
Epoch 980/10000
1/1 [==============================] - 0s 14ms/step - loss: 5567.2271 - val_loss: 14724.8779
Epoch 981/10000
1/1 [==============================] - 0s 14ms/step - loss: 5801.1572 - val_loss: 13099.2490
Epoch 982/10000
1/1 [==============

Epoch 1048/10000
1/1 [==============================] - 0s 14ms/step - loss: 5453.4980 - val_loss: 12733.7959
Epoch 1049/10000
1/1 [==============================] - 0s 14ms/step - loss: 6008.7134 - val_loss: 13169.5547
Epoch 1050/10000
1/1 [==============================] - 0s 13ms/step - loss: 5680.0952 - val_loss: 12545.8516
Epoch 1051/10000
1/1 [==============================] - 0s 14ms/step - loss: 5555.6318 - val_loss: 12534.9062
Epoch 1052/10000
1/1 [==============================] - 0s 14ms/step - loss: 5504.8101 - val_loss: 12507.6797
Epoch 1053/10000
1/1 [==============================] - 0s 14ms/step - loss: 5579.5938 - val_loss: 12592.5908
Epoch 1054/10000
1/1 [==============================] - 0s 14ms/step - loss: 5470.4580 - val_loss: 12512.3057
Epoch 1055/10000
1/1 [==============================] - 0s 14ms/step - loss: 5538.0742 - val_loss: 12592.3408
Epoch 1056/10000
1/1 [==============================] - 0s 14ms/step - loss: 5567.6167 - val_loss: 12608.4834
Epoch 1057

1/1 [==============================] - 0s 14ms/step - loss: 5327.5728 - val_loss: 14225.2646
Epoch 1123/10000
1/1 [==============================] - 0s 14ms/step - loss: 5697.0332 - val_loss: 14581.4346
Epoch 1124/10000
1/1 [==============================] - 0s 14ms/step - loss: 5738.6738 - val_loss: 14004.5732
Epoch 1125/10000
1/1 [==============================] - 0s 14ms/step - loss: 5977.4478 - val_loss: 14025.9463
Epoch 1126/10000
1/1 [==============================] - 0s 14ms/step - loss: 5608.3130 - val_loss: 14172.9541
Epoch 1127/10000
1/1 [==============================] - 0s 14ms/step - loss: 5652.1851 - val_loss: 14120.0752
Epoch 1128/10000
1/1 [==============================] - 0s 14ms/step - loss: 5978.2383 - val_loss: 14194.4189
Epoch 1129/10000
1/1 [==============================] - 0s 14ms/step - loss: 5631.0610 - val_loss: 13556.6592
Epoch 1130/10000
1/1 [==============================] - 0s 14ms/step - loss: 5788.0508 - val_loss: 13091.5312
Epoch 1131/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 5565.3232 - val_loss: 13388.0000
Epoch 1197/10000
1/1 [==============================] - 0s 14ms/step - loss: 5296.3008 - val_loss: 12349.2725
Epoch 1198/10000
1/1 [==============================] - 0s 14ms/step - loss: 6060.7861 - val_loss: 13394.6924
Epoch 1199/10000
1/1 [==============================] - 0s 22ms/step - loss: 5806.1943 - val_loss: 13098.1748
Epoch 1199: early stopping


In [20]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 48ms/step
[811.86](test_target) - [[632.25275]](prediction) = 179.60725341796876


In [21]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [22]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Rio Grande do Norte - IDH,Rio Grande Do Norte - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Norte - Desemprego,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado
0,-1.711016,-1.058842,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-0.707325,...,-2.053056,1.054559,1.834095,0.284160,-0.831133,1.848182,-1.600293,-1.906391,-2.085127,-1.929514
1,-1.108077,-0.907127,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.408923,...,-1.157660,1.481313,0.974292,-1.481726,-0.325790,1.004081,-1.054056,-0.980456,-0.930471,-0.985258
2,-0.564023,-0.693649,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.464423,...,0.017300,1.117907,0.483020,-1.426495,1.105095,0.765040,-0.556001,-0.547259,-0.038487,-0.249104
3,-0.077377,-0.308093,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,0.700578,...,0.547811,-0.119980,-0.819191,-0.790718,1.785989,-0.325730,-0.117531,0.015518,0.473329,0.244404
4,0.353338,-0.014792,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,1.641423,...,0.778022,-1.174513,-1.151912,0.706515,0.719279,-0.598927,0.258577,0.653770,0.644220,0.510165
5,0.729600,0.204541,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,1.230846,...,0.876253,-0.721714,-0.172008,1.088878,-0.448496,-0.985782,0.548105,0.634290,0.438431,0.441223
6,1.052885,0.500355,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,-0.265141,...,0.869751,-0.511919,0.045000,0.993533,-0.824042,-0.632652,0.781240,1.013615,0.104248,0.354425
7,1.324670,2.277607,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,-0.727035,...,0.121579,-1.125653,-1.193296,0.625853,-1.180902,-1.074212,1.739960,1.116913,1.393857,1.613659


train_target:


,Rio Grande Do Norte - Consumo de Cimento (t)
0,404.604
1,450.141
2,526.789
3,602.047
4,705.791
5,694.478
6,850.190
7,836.975


test_input:


...

,Rio Grande do Norte - IDH,Rio Grande Do Norte - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande do Norte - Desemprego,Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado
17,1.048153,0.793617,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-1.927738,...,-0.902462,2.176045,2.06131,1.506174,-1.074664,0.939491,0.751768,-1.131139,-0.273722,0.314748


test_target:


,Rio Grande Do Norte - Consumo de Cimento (t)
17,811.86


1/1 [==============================] - 0s 35ms/step
Error: 179.60725341796876




[791.9650268554688,
 810.1727905273438,
 907.9131469726562,
 917.7420654296875,
 786.8482055664062,
 796.7445678710938,
 776.6544799804688,
 591.3558349609375,
 655.8670654296875,
 632.2527465820312]

In [23]:
display(targets)
display(predictions)

[961.431,
 954.17,
 886.1657142857142,
 802.5085714285714,
 730.2617142857143,
 646.482,
 485.012,
 563.293,
 744.8629999999999,
 811.86]

[791.9650268554688,
 810.1727905273438,
 907.9131469726562,
 917.7420654296875,
 786.8482055664062,
 796.7445678710938,
 776.6544799804688,
 591.3558349609375,
 655.8670654296875,
 632.2527465820312]

In [24]:
mae = mean_absolute_error(predictions, targets)
mae

124.56016713867189

In [25]:
porcentage = mae/np.mean(targets)
porcentage

0.16419640840436642